In [1]:
import andes
import os
import numpy as np
import csv

import matplotlib
%matplotlib inline

Example code for batch data generation:

https://docs.andes.app/en/latest/_examples/ex6.html

# Define case path

In [2]:
andes.config_logger(stream_level=50)
# 10-DEBUG，20-INFO30WARNING，40-ERROR，50-CRITICAL

# case path
dir_path = os.path.abspath('..')
loc_path_case = '/Batch_data_gen_NN_train/case123_REGCV1_v5.xlsx'

case = dir_path + loc_path_case

# Generate settings data

### Input data:

**sf_load**: load scaling factor <br> Range: [0.5, 2.5]

**p0_gen**: p0 for generators <br> [SG1, SG2, IBR1, IBR2, IBR3, IBR4, IBR5, IBR6, IBR7, IBR8] <br> Reange for each SG: [0, 0.5], Reange for each IBR: [0, 0.05]

**m_ibr**: virtual inertia for REGCV1 <br> [M1, M2, M3, M4, M5, M6, M7, M8], Range for each M: [0, 6]

**n_ibr**: virtual damping for REGCV1 <br> [D1, D2, D3, D4, D5, D6, D7, D8], Range for each M: [0, 4]

**input** = [m_ibr, n_ibr]

In [3]:
import numpy as np

# Define the number of data points you want
num_samples = 1000  # You can change this as needed

# Define the ranges for each variable
sf_load_range = [0.9, 1.1]
p0_gen_sg_range = [0.1, 0.5]
p0_gen_ibr_range = [0, 0.05]
m_ibr_range = [0, 6]
n_ibr_range = [0, 4]

# Initialize an empty list to store valid data points
input     = []
input_new = []

# Generate random data until you have enough valid points
for _ in range(num_samples):
    # Generate random values for each variable
    sf_load = np.random.uniform(sf_load_range[0], sf_load_range[1])
    sf_load = np.array([sf_load])
    
    # Generate random values for p0_gen with constraints
    p0_gen_sg = np.random.uniform(p0_gen_sg_range[0], p0_gen_sg_range[1], 2)
    p0_gen_ibr = np.random.uniform(p0_gen_ibr_range[0], p0_gen_ibr_range[1], 8)
    p0_gen = np.concatenate((p0_gen_sg, p0_gen_ibr))
    p0_gen = p0_gen/np.sum(p0_gen)*3.53/10*0.9 # 3.53 is base load, 10 to change Sbase from andes to previous 100MVA
                                               # 0.9 is the scaling factor, leave 0.1 slack bus

    # Generate random values for m_ibr and n_ibr
    m_ibr = np.random.uniform(m_ibr_range[0], m_ibr_range[1], 8)
    n_ibr = np.random.uniform(n_ibr_range[0], n_ibr_range[1], 8)
    
    # Combine all the data into one array
    combine     = np.concatenate((sf_load, p0_gen, m_ibr, n_ibr))
    combine_new = np.concatenate((m_ibr, n_ibr))

    # Append the generated data to the valid_data list
    input.append(combine)
    input_new.append(combine_new)


# Batch data generation

### Output data

**rocof_max**: Hz/s, maximum rate of change of frequency

**fnadir** Hz, lowest nadir deviation

**dtheta_max**: rad, maximum angle difference

**busV_min**: pu, minmum voltage

**eig_max**: maximum eigenvalue before gen trip

**eig_max_**: maximum eigenvalue after gen trip

eig_max = max (eig_max, eig_max_)

**Output** = [rocof_max, fnadir, dtheta_max, eig_max]

In [4]:
output = []

for i in range(num_samples):
    
    # ------------ Reload case -------------
    # --------------------------------------
    ssa = andes.load(case,
                 setup=True,
                 no_output=True)

    # ------- Modify case parameters -------
    # --------------------------------------
    # reset load
    ssa.PQ.set(
                src='p0', 
                idx= ssa.PQ.idx.v, 
                attr='v',
                value= ssa.PQ.p0.v * input[i][0]
            )
    # reset power setting point
    ssa.PV.set(
                src='p0', 
                idx= ssa.PV.idx.v, 
                attr='v',
                value= input[i][1]
            )
    # reset REGCV1 control parameters
    ssa.REGCV1.set(
                    src='M', 
                    idx= ssa.REGCV1.idx.v,
                    attr='v',
                    value= input[i][2]
                )
    ssa.REGCV1.set(
                    src='D', 
                    idx= ssa.REGCV1.idx.v, 
                    attr='v',
                    value= input[i][3]
                )

    # ------- Run simulation ---------------
    # --------------------------------------
    ssa.PFlow.run()
    ssa.TDS.init()

    ssa.EIG.run()
    eig     = ssa.EIG.calc_eig()
    eig_max = np.max(eig[0].real)

    # Turn off stability check in TDS
    ssa.TDS.config.criteria = 0
    ssa.TDS.config.no_tqdm = 1    # Turn off simulation progress bar
    ssa.TDS.config.tf = 10        # Simulation time
    ssa.TDS.run()
    ssa.EIG.run()

    eig_     = ssa.EIG.calc_eig()
    eig_max_ = np.max(eig[0].real)

    # ------- extract data from TDS-----------------
    # --------------------------------------
    # Extract time
    t = ssa.dae.ts.t
    t = t[:, np.newaxis]

    # Extract RoCoF
    rocof     = ssa.dae.ts.y[:,  ssa.BusROCOF.Wf_y.a]
    rocof_max = np.max(np.abs(rocof*60))

    # Extract frequency
    f         = ssa.dae.ts.x[:,  ssa.GENROU.omega.a]
    fnadir    = np.min(f)*60 - 60

    # Extract voltage
    # busV      = ssa.dae.ts.y[:,  ssa.Bus.v.a]
    # busV_min  = np.min(busV)

    # Extract power angle
    theta      = ssa.dae.ts.y[:, ssa.GENROU.a.a]
    dtheta     = np.max(theta, axis=1) - np.min(theta, axis=1)
    dtheta_max = np.max(dtheta)

    output.append([rocof_max, fnadir, dtheta_max, np.max([eig_max, eig_max_])])

<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.


/home/buxin/andes/andes/routines/eig.py:222: LinAlgWarning: Ill-conditioned matrix (rcond=9.38493e-20): result may not be accurate.
  WT = solve(N, Weye, overwrite_b=True)


<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.


/home/buxin/andes/andes/routines/eig.py:222: LinAlgWarning: Ill-conditioned matrix (rcond=2.87894e-17): result may not be accurate.
  WT = solve(N, Weye, overwrite_b=True)


<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.


...

<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.


/home/buxin/andes/andes/routines/eig.py:222: LinAlgWarning: Ill-conditioned matrix (rcond=1.95472e-17): result may not be accurate.
  WT = solve(N, Weye, overwrite_b=True)


<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.


/home/buxin/andes/andes/routines/eig.py:222: LinAlgWarning: Ill-conditioned matrix (rcond=8.89017e-17): result may not be accurate.
  WT = solve(N, Weye, overwrite_b=True)


<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: GENROU.GENROU_2 status changed to 0 at t=1.0 sec.
<Toggle Toggler_1>: 

# Save data

Or save data in the loop

In [5]:
filename = "input_1000.csv"

# Save the list to a CSV file
with open(filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(input_new)

print("Input Data saved to", filename)

Input Data saved to input_1000.csv


In [6]:
filename = "output_1000.csv"

# Save the list to a CSV file
with open(filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(output)

print("Output Data saved to", filename)

Output Data saved to output_1000.csv
